In [4]:
import configparser
import csv
import json
import math
import os
import pickle
import platform
import sqlite3
import time
import itertools
import math

import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.path import Path
from matplotlib.patches import ConnectionStyle
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import gaussian_kde
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.path import Path
from matplotlib.patches import ConnectionStyle, Polygon
from matplotlib.collections import PatchCollection
from matplotlib import collections


try:
    from importlib import metadata as importlib_metadata
except ImportError:
    # Backwards compatibility - importlib.metadata was added in Python 3.8
    import importlib_metadata

from PySide6.QtGui import QAction, QGuiApplication
from PySide6.QtWidgets import QAbstractItemView, QMainWindow, QApplication, QMenu, QToolBar, QFileDialog, QTableView, QVBoxLayout, QHBoxLayout, QWidget, QSlider,  QGroupBox , QLabel , QWidgetAction, QPushButton, QSizePolicy
from PySide6.QtCore import QAbstractTableModel, QModelIndex, QVariantAnimation, Qt


from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import pandas as pd
from PySide6.QtGui import QGuiApplication

from PySide6.QtCore import QAbstractTableModel, Qt, QModelIndex
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] =  'truetype'



In [1]:


# Function to monitor the download progress
start_time = time.time()
def download_progress(count, block_size, total_size):
    global start_time
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    print(f"Downloaded {progress_size} of {total_size} bytes ({percent}% done), speed {speed} KB/s")
    if progress_size >= total_size:  # reset start_time for next download
        start_time = time.time()

# Function to download and extract a zip file
def download_and_extract(target_dir, repo_link, date):
    zip_file = target_dir+".zip"
    url = quote(repo_link+"/raw/main/"+date + "/"+zip_file, safe='/:')

    if not os.path.exists(target_dir):
        if os.path.exists(zip_file):
            extract_dir = os.path.splitext(zip_file)[0]
            os.makedirs(extract_dir, exist_ok=True)

            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                total_files = len(zip_ref.infolist())
                extracted_files = 0
                for file in zip_ref.infolist():
                    zip_ref.extract(file, path=extract_dir)
                    extracted_files += 1
                    print(f'Unzip Progress: {extracted_files / total_files * 100:.2f}%')
        else:
            urllib.request.urlretrieve(url, zip_file, reporthook=download_progress)
            extract_dir = os.path.splitext(zip_file)[0]
            os.makedirs(extract_dir, exist_ok=True)

            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                total_files = len(zip_ref.infolist())
                extracted_files = 0
                for file in zip_ref.infolist():
                    zip_ref.extract(file, path=extract_dir)
                    extracted_files += 1
                    print(f'Unzip Progress: {extracted_files / total_files * 100:.2f}%')

        print(f'{zip_file} has been downloaded and extracted to \n {os.path.abspath(extract_dir)}')

    # Open the directory in the file explorer
    if platform.system() == "Windows":
        os.startfile(os.path.abspath(extract_dir))
    elif platform.system() == "Darwin":
        os.system(f'open "{os.path.abspath(extract_dir)}"')
    else:
        os.system(f'xdg-open "{os.path.abspath(extract_dir)}"')

# Function to fix a line in a CSV file
def fix_line(line):
    pattern = r'(?<!")"(?!")'
    fixed_line = re.sub(pattern, '', line)
    return fixed_line

# Function to fix a CSV file
def fix_csv_file(file_path):
    with open(file_path, 'r', newline='', encoding='ISO-8859-1') as f:
        reader = csv.reader(f)
        rows = list(reader)

    has_extra_quotes = any('"' in field for row in rows for field in row)

    if not has_extra_quotes:
        return
    else:
        print(f'File "{file_path}" has isolated quotes error.')
        

    with open(file_path, 'w', newline='', encoding='ISO-8859-1') as f:
        writer = csv.writer(f)
        for row in rows:
            fixed_row = [fix_line(field) for field in row]
            writer.writerow(fixed_row)

# Download the data file    
target_dir = "GEOROC Compilation Rock Types"
repo_link = "https://github.com/GeoPyTool/GeoRocData"
date = "2023-12-01"
download_and_extract(target_dir, repo_link, date)

# Fix Quota Error in the downloaded CSV files
files = os.listdir(target_dir)
for file in files:
    file_name = os.path.join(target_dir, file)
    _, extension = os.path.splitext(file_name)
    if extension.lower() == '.csv':
        fix_csv_file(file_name)

c:\Anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU RAM Free: 7957MB | Used: 0MB | Util   0% | Total 8188MB


In [4]:

# Funtion to split the CSV file into three separate files
def split_csv(file_path):

    # Split the path and filename
    dir_path, file_name = os.path.split(file_path)

    print("Dir Path:", dir_path)
    print("Filename:", file_name)

    # Get the parent directory of output_dir
    parent_dir = os.path.dirname(dir_path)

    # Join the parent directory and the 'Splited' folder
    splited_data_dir = os.path.join(parent_dir, 'Splited_Data')
    splited_abbreviations_dir = os.path.join(parent_dir, 'Splited_Abbreviations')
    splited_references_dir = os.path.join(parent_dir, 'Splited_References')

    # Check if splited_dir exists, if not, create it
    for i in [splited_data_dir, splited_abbreviations_dir, splited_references_dir]:
        if not os.path.exists(i):
            os.makedirs(i)

    # Open the file and read the lines
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        lines = f.readlines()

    # Find the start and end lines for each section
    start_line_main = 0
    end_line_main = next(i for i, line in enumerate(lines) if 'Abbreviations:' in line) - 2
    start_line_abbreviations = end_line_main + 2
    end_line_abbreviations = next(i for i, line in enumerate(lines) if 'References:' in line) - 2
    start_line_references = end_line_abbreviations + 2
    end_line_references = len(lines) - 1

    # Read each section into a DataFrame    
    df_data = pd.read_csv(file_path, skiprows=start_line_main, nrows=end_line_main-start_line_main, encoding='ISO-8859-1', engine='python', on_bad_lines='warn')
    df_abbreviations = pd.read_csv(file_path, skiprows=start_line_abbreviations, nrows=end_line_abbreviations-start_line_abbreviations, encoding='ISO-8859-1', engine='python', on_bad_lines='warn')
    df_references = pd.read_csv(file_path, skiprows=start_line_references, nrows=end_line_references-start_line_references, encoding='ISO-8859-1', engine='python', on_bad_lines='warn')

    # Write each DataFrame to a separate CSV file
    df_data.to_csv(splited_data_dir+'/'+file_name.replace('.csv','')+'_data.csv', index=False)
    df_abbreviations.to_csv(splited_abbreviations_dir+'/'+file_name.replace('.csv','')+ '_abbreviations.csv', index=False)
    df_references.to_csv(splited_references_dir+'/'+file_name.replace('.csv','')+ '_references.csv', index=False)

    # Open the directory in the file explorer
    if platform.system() == "Windows":
        os.startfile(os.path.abspath(splited_data_dir))
    elif platform.system() == "Darwin":
        os.system(f'open "{os.path.abspath(splited_data_dir)}"')
    else:
        os.system(f'xdg-open "{os.path.abspath(splited_data_dir)}"')

# Set the target directory name
target_dir = 'GEOROC Compilation Rock Types'

# Get all files in target_dir
files = os.listdir(target_dir)

# For each file in target_dir, run split_csv
for file in files:
    file_name = os.path.join(target_dir, file)
    # Get the file extension
    _, extension = os.path.splitext(file_name)
    # If the file is a CSV file, run split_csv
    if extension.lower() == '.csv':
        split_csv(file_name)

In [5]:


# Set the target directory name
target_dir = 'Splited_Data'

# Add the filename as a new column to the CSV file
def add_filename_as_column(file_path):    
    """
    This function adds the filename as a new column to the CSV file.
    """

    # Split the file path into directory path and file name
    dir_path, file_name = os.path.split(file_path)

    print("Directory Path:", dir_path)
    print("Filename:", file_name)

    # Get the parent directory of the output_dir
    parent_dir = os.path.dirname(dir_path)

    # Join the parent directory with the 'Type_Added' folder
    output_dir = os.path.join(parent_dir, 'Type_Added')

    # Check if output_dir exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a new file path for the modified data
    output_path = os.path.join(output_dir, file_name.replace('.csv', '') + '_type_added.csv')

    # Process the filename
    split_file = file_name.split('_')
    if len(split_file) > 1:
        processed_file = split_file[1].capitalize()
    else:
        pass  # Skip this file if it doesn't contain an underscore

    # Remove '.csv' from the processed filename
    processed_file = processed_file.replace('.csv', '')

    # Try to read the file with different encodings
    try:
        data = pd.read_csv(file_path, encoding='utf-8', engine='python', on_bad_lines='warn')
        encoding='utf-8'
    except UnicodeDecodeError:
        try:
            data = pd.read_csv(file_path, encoding='ISO-8859-1', engine='python', on_bad_lines='warn')
            encoding='ISO-8859-1'
        except UnicodeDecodeError:
            data = pd.read_csv(file_path, encoding='Windows-1252', engine='python', on_bad_lines='warn')
            encoding='Windows-1252'

    # Check if the first column is already 'Type'
    if data.columns[0] != 'Type':
        # Add 'Type' to the first column
        data.insert(0, 'Type', processed_file)

    # Write the data back to a new file
    data.to_csv(output_path, index=False, encoding=encoding)

# Get all files in target_dir
files = os.listdir(target_dir)

# For each file in target_dir, run split_csv
for file in files:
    file_name = os.path.join(target_dir, file)
    # Get the file extension
    _, extension = os.path.splitext(file_name)
    # If the file is a CSV file, run split_csv
    if extension.lower() == '.csv':
        add_filename_as_column(file_name)

"Sure, here is a joke for you:\n\nWhat do you call a boomerang that won't come back?\n\nA stick."

In [6]:

# Set the target directory and database names
target_dir = 'Type_Added'
db_name = 'GeoRoc.db'

def csv_to_db_with_check(target_dir, db_name):
    """
    This function reads CSV files from the target directory and writes them to a SQLite database.
    It also checks for and removes duplicate rows in the CSV files.
    """

    # Create a set to store the paths of files that have been read
    read_files = set()

    # Create a SQLite database
    conn = sqlite3.connect(db_name)

    # Open the record file and read the paths of files that have been read
    with open(db_name.replace('.db', '_') + 'imported_files.txt', 'a+') as f:
        f.seek(0)
        read_files = set(line.strip() for line in f)

    # Iterate over all files in the target directory
    for file in os.listdir(target_dir):
        # Get the full path of the file
        file_path = os.path.join(target_dir, file)

        # If the file has been read, skip it
        if file_path in read_files:
            continue

        # Get the extension of the file
        _, extension = os.path.splitext(file_path)

        # If the file is a CSV file
        if extension.lower() == '.csv':
            # Try to read the file with different encodings
            try:
                data = pd.read_csv(file_path, encoding='utf-8', engine='python', on_bad_lines='warn')
            except UnicodeDecodeError:
                try:
                    data = pd.read_csv(file_path, encoding='ISO-8859-1', engine='python', on_bad_lines='warn')
                except UnicodeDecodeError:
                    data = pd.read_csv(file_path, encoding='Windows-1252', engine='python', on_bad_lines='warn')

            # Check for duplicates
            print("Duplicates before removal: ", data.duplicated().sum())

            # Remove duplicates
            data = data.drop_duplicates()

            # Check for duplicates again
            print("Duplicates after removal: ", data.duplicated().sum())

            # Write the data to the SQLite database
            data.to_sql('Current_Data', conn, if_exists='append', index=False)

            # Record the path of the file that has been read
            with open(db_name.replace('.db', '_') + 'imported_files.txt', 'a') as f:
                f.write(file_path + '\n')

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

# Call the function
csv_to_db_with_check(target_dir, db_name)

In [7]:


# Set the target directory and database names
target_dir = 'Splited_References'
db_name = 'Ref.db'

def csv_to_db_with_check(target_dir, db_name):
    """
    This function reads CSV files from the target directory and writes them to a SQLite database.
    """

    # Create a set to store the paths of files that have been read
    read_files = set()

    # Create a SQLite database
    conn = sqlite3.connect(db_name)

    # Open the record file and read the paths of files that have been read
    with open(db_name.replace('.db', '_') + 'imported_files.txt', 'a+') as f:
        f.seek(0)
        read_files = set(line.strip() for line in f)

    # Iterate over all files in the target directory
    for file in os.listdir(target_dir):
        # Get the full path of the file
        file_path = os.path.join(target_dir, file)

        # If the file has been read, skip it
        if file_path in read_files:
            continue

        # Get the extension of the file
        _, extension = os.path.splitext(file_path)

        # If the file is a CSV file
        if extension.lower() == '.csv':
            # Try to read the file with different encodings
            try:
                data = pd.read_csv(file_path, encoding='utf-8', engine='python', on_bad_lines='warn', skiprows=1, header=None)
            except UnicodeDecodeError:
                try:
                    data = pd.read_csv(file_path, encoding='ISO-8859-1', engine='python', on_bad_lines='warn', skiprows=1, header=None)
                except UnicodeDecodeError:
                    data = pd.read_csv(file_path, encoding='Windows-1252', engine='python', on_bad_lines='warn', skiprows=1, header=None)

            # Write the data to the SQLite database
            data.to_sql('Current_Data', conn, if_exists='append', index=False)

            # Record the path of the file that has been read
            with open(db_name.replace('.db', '_') + 'imported_files.txt', 'a') as f:
                f.write(file_path + '\n')

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

# Call the function
csv_to_db_with_check(target_dir, db_name)

In [8]:
# Create a figure with aspect ratio 2:1
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)

# Set the aspect ratio of axes to 3:2
ax.set_aspect(3/2)

start_time = time.time()


def download_progress(count, block_size, total_size):
    """
    This function is used to track the progress of a download.
    """
    global start_time
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    print(f"Downloaded {progress_size} of {total_size} bytes ({percent}% done), speed {speed} KB/s")
    if progress_size >= total_size:  # reset start_time for next download
        start_time = time.time()


def download_and_extract(filename, zip_file, url, extract_dir):
    """
    This function downloads a zip file from a given URL, extracts it, and opens the directory in the file explorer.

    Args:
    filename (str): The name of the file to check if it exists.
    zip_file (str): The name of the zip file to download.
    url (str): The URL to download the zip file from.
    extract_dir (str): The directory to extract the zip file to.
    """
    if not os.path.exists(filename):
        if os.path.exists(zip_file):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                total_files = len(zip_ref.infolist())
                extracted_files = 0
                for file in zip_ref.infolist():
                    zip_ref.extract(file, path=extract_dir)
                    extracted_files += 1
                    print(f'Unzip Progress: {extracted_files / total_files * 100:.2f}%')
        else:
            urllib.request.urlretrieve(url, zip_file, reporthook=download_progress)
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                total_files = len(zip_ref.infolist())
                extracted_files = 0
                for file in zip_ref.infolist():
                    zip_ref.extract(file, path=extract_dir)
                    extracted_files += 1
                    print(f'Unzip Progress: {extracted_files / total_files * 100:.2f}%')

        print(f'{zip_file} has been downloaded and extracted to \n {os.path.abspath(extract_dir)}')

        # Open the directory in the file explorer
        if platform.system() == "Windows":
            os.startfile(os.path.abspath(extract_dir))
        elif platform.system() == "Darwin":
            os.system(f'open "{os.path.abspath(extract_dir)}"')
        else:
            os.system(f'xdg-open "{os.path.abspath(extract_dir)}"')


def visualize_data(df, name):
    """
    This function is used to visualize the data.
    """
    data = (df[(df["ROCK TYPE"] == name)]['Type'].value_counts())

    # Set the golden ratio
    golden_ratio = 1.618

    # Set the width and height of the figure
    fig_width = 10
    fig_height = fig_width / golden_ratio

    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Sort the data
    data_sorted = data.sort_values(ascending=False)

    # Calculate the median of the sample quantity
    median = data_sorted.median()

    # Draw a horizontal line to represent the median using the axes method
    ax.axhline(y=median, color='k', linestyle='--')

    # Create a vertical bar chart using the axes method
    colors = ['black' if x >= median else 'gray' for x in data_sorted.values]
    bars = ax.bar(data_sorted.index, data_sorted.values, color=colors, alpha=0.3)

    # Add the corresponding value at the highest point inside each bar
    for bar in bars:
        height = bar.get_height()
        if height >= median:
            ax.text(bar.get_x() + bar.get_width() / 2, height,
                    '{:d}'.format(int(height)), ha='center', va='bottom', rotation=90)

    # Find the index of the first value below the median
    first_below_median_index = next(i for i, x in enumerate(data_sorted.values) if x < median)

    # Get the corresponding x value
    first_below_median_x = data_sorted.index[first_below_median_index]

    # Add the value of the median next to the median line
    ax.text(first_below_median_x, median, 'Median: {:.2f}'.format(median), color='k', va='bottom')

    # Set the color of the x-axis labels
    for ticklabel, tickcolor in zip(ax.get_xticklabels(), colors):
        ticklabel.set_color(tickcolor)

    # Find the indices of all values greater than 30
    indices_gt_30 = [i for i, x in enumerate(data_sorted.values) if x > 30]

    # Display these indices and their corresponding values in the upper right part of the graph, divided into three columns
    for idx, i in enumerate(indices_gt_30):
        color = 'black' if data_sorted.values[i] >= median else 'gray'
        if idx % 3 == 0:  # For indices with a modulus of 0, put them in the left column
            ax.text(0.33, 1 - 0.05 * (idx // 3) - 0.1, ' {}: {}'.format(data_sorted.index[i], data_sorted.values[i]),
                    color=color, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
        elif idx % 3 == 1:  # For indices with a modulus of 1, put them in the middle column
            ax.text(0.66, 1 - 0.05 * (idx // 3) - 0.1, ' {}: {}'.format(data_sorted.index[i], data_sorted.values[i]),
                    color=color, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
        else:  # For indices with a modulus of 2, put them in the right column
            ax.text(1, 1 - 0.05 * (idx // 3) - 0.1, ' {}: {}'.format(data_sorted.index[i], data_sorted.values[i]),
                    color=color, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)

    # Add title and labels using the axes method
    ax.set_title('Data Count of ' + name + ' Type')
    ax.set_ylabel('Number of Data Entries')

    # Set the right limit of the x-axis
    ax.set_xlim(right=data_sorted.index[-1])

    # Rotate the labels of the x-axis and y-axis
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=90)

    # Remove the top and right frames of the graph
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Adjust the figure automatically
    fig.tight_layout()

    # Save the figure as SVG and PNG files
    output_dir = 'Db_Stats'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    fig.savefig(output_dir + '/' + name + '_Stats.svg')
    fig.savefig(output_dir + '/' + name + '_Stats.jpg', dpi=600)
    data.to_csv(output_dir + '/' + name + '_List.csv')


# Define the filename, zip file, URL, and extraction directory
filename = "GeoRoc.db"
zip_file = "GeoRoc.zip"
url = "https://github.com/GeoPyTool/GeoRocData/raw/main/GeoRoc.zip"
extract_dir = "./"

# Download and extract the file
download_and_extract(filename, zip_file, url, extract_dir)

# Connect to the database
conn = sqlite3.connect(filename)

# Read the data from the Current_Data table
df = pd.read_sql_query("SELECT * FROM Current_Data", conn)

# Visualize the data
visualize_data(df, name='VOL')
visualize_data(df, name='PLU')

# Close the connection to the database
conn.close()

"Sure, Langsmith can be a valuable tool for testing, especially for automating repetitive tasks and identifying potential issues. Here are some ways Langsmith can help with testing:\n\n**1. Identifying Test Cases:**\n\n* **Natural Language Description:** You can define test cases in plain English using the Langsmith user interface. \n* **Test Data Integration:** You can integrate test data directly into the test cases, eliminating manual data entry and ensuring accuracy.\n* **Conditional Testing:** You can create conditional statements to specify specific test scenarios based on various conditions.\n\n**2. Test Case Execution and Reporting:**\n\n* **Test Case Execution:** Langsmith can run test cases automatically, saving you time and effort.\n* **Test Case Reporting:** Langsmith generates comprehensive reports that highlight test results, pass/fail status, and detailed logs.\n\n**3. Identifying Potential Issues:**\n\n* **Error Detection:** Langsmith identifies syntax errors, logical e

In [ ]:

def generate_harker_diagram(filename, rock_type, major_oxides,output_dir):
    # Connect to the database
    conn = sqlite3.connect(filename)

    # Read the data from the Current_Data table
    df = pd.read_sql_query("SELECT * FROM Current_Data", conn)

    # Select columns
    selected_columns = df[["Type", "SIO2(WT%)", "TIO2(WT%)", "AL2O3(WT%)", "FEOT(WT%)",  "CAO(WT%)", 
                    "MGO(WT%)", "MNO(WT%)", "K2O(WT%)", "NA2O(WT%)", "P2O5(WT%)","ROCK TYPE"]]

    # Filter rows where "ROCK TYPE" is rock_type and "SIO2(WT%)" is not 0
    tag_df = selected_columns[(selected_columns["ROCK TYPE"] == rock_type) & (selected_columns["SIO2(WT%)"] != 0)]

    # Output the count of "Type" values
    print(tag_df["Type"].value_counts())
    type_counts = tag_df["Type"].value_counts()
    type_counts.to_csv(output_dir + '/'+'Harker_'+rock_type+'.csv')

    # Create bi-variant plots
    fig, axes = plt.subplots(3, 3, figsize=(9, 9))

    # Check if tag_color_dict.json file exists
    # 检查是否存在tag_color_dict.json文件
    if os.path.exists('Color_Config/'+rock_type+'_color_dict.json'):
        # 如果存在，从文件中读取tag_color_dict
        with open('Color_Config/'+rock_type+'_color_dict.json', 'r') as f:
            tag_color_dict = json.load(f)
    else:
        # 如果不存在，创建新的tag_color_dict并保存到文件中
        type_set = set(tag_df['Type'].unique())
        cmap = cm.get_cmap('rainbow', len(type_set))
        tag_color_dict = {type: cmap(i) for i, type in enumerate(type_set)}
        with open('Color_Config/'+rock_type+'_color_dict.json', 'w') as f:
            json.dump(tag_color_dict, f) 


    # Draw Harker scatter plots
    for i, oxide in enumerate(major_oxides):
        # Determine position on the subplot grid
        row = i // 3
        col = i % 3

        # Create scatter plot
        grouped = tag_df.groupby('Type')
        for label, group in grouped:
            axes[row, col].scatter(group ['SIO2(WT%)'], group [oxide], alpha=0.15, label=label, color=tag_color_dict[label], edgecolors='none')

        newoxide =  oxide.replace('(WT%)',' wt%')
        axes[row, col].set_xlabel('SiO2 wt%', fontsize=7)
        axes[row, col].set_ylabel(f'{newoxide}', fontsize=7)
        ax = axes[row, col]

        # Calculate the 1% and 9999% quantiles of the y-axis data
        q1 = tag_df[oxide].quantile(0.01)
        q3 = tag_df[oxide].quantile(0.9999)
        # Calculate the interquartile range
        iqr = q3 - q1
        # Calculate the most concentrated range
        lower_bound = min(tag_df[oxide])
        upper_bound = q3 + 0.1 * iqr

        # Limit the y-axis to the most concentrated range
        if not np.isnan(lower_bound) and not np.isnan(upper_bound) and not np.isinf(lower_bound) and not np.isinf(upper_bound):
            axes[row, col].set_ylim(lower_bound, upper_bound)
        else:
            print(f"Invalid y-axis limits for row {row}, column {col}: lower_bound={lower_bound}, upper_bound={upper_bound}")

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

        # Calculate the number of data points within the view range
        visible_points = tag_df[(tag_df['SIO2(WT%)'] >= xlim[0]) & (tag_df['SIO2(WT%)'] <= xlim[1]) & 
                                (tag_df[oxide] >= ylim[0]) & (tag_df[oxide] <= ylim[1])]

        num_visible_points = len(visible_points)

        # Display the number of visible data points on the plot
        ax.text(0.05, 0.95, f'Visible points: {num_visible_points}', transform=ax.transAxes, verticalalignment='top')

    fig.tight_layout()

    # Create a legend on the right side of the whole figure
    legend = fig.legend(loc='center left', fontsize=4, bbox_to_anchor=(1, 0.5))
    # Save the plot, including the legend
    plt.savefig(output_dir + '/'+'Harker_'+rock_type+'.svg')
    plt.savefig(output_dir + '/'+'Harker_'+rock_type+'.png', dpi=600)

    conn.close()


# Set the parameters
filename = 'GeoRoc.db'
rock_type = 'VOL'
color_config_dir = 'Color_Config'
if not os.path.exists(color_config_dir):
    os.makedirs(color_config_dir)
output_dir = 'Harker'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# List of major oxides for y-axis
major_oxides = ["TIO2(WT%)", "AL2O3(WT%)", "FEOT(WT%)",  "CAO(WT%)", 
                "MGO(WT%)", "MNO(WT%)", "K2O(WT%)", "NA2O(WT%)", "P2O5(WT%)"]
# generate_harker_diagram(filename, rock_type, major_oxides)
generate_harker_diagram(filename,'VOL', major_oxides,output_dir)
generate_harker_diagram(filename,'PLU', major_oxides,output_dir)

In [ ]:

with open(current_directory+'/Plot_Json/tas_cord.json', 'r', encoding='utf-8') as file:
    cord = json.load(file)

# 创建一个宽高比为2:1的figure
fig = plt.figure(figsize=(10, 5))     
ax = fig.add_subplot(1, 1, 1)
# 设置axes的宽高比为3:2
ax.set_aspect(3/2)

def TAS_all(filename, rock_type, output_dir):
    # 连接到数据库
    conn = sqlite3.connect(filename)

    # Read the data from the TAS_Data table
    df = pd.read_sql_query("SELECT * FROM Current_Data", conn)

    selected_columns = df[["Type", "SIO2(WT%)", "NA2O(WT%)", "K2O(WT%)","ROCK TYPE"]]

    # 筛选'ROCK TYPE'为tag的行，并且去掉不含SiO2的行
    tag_df = selected_columns[(selected_columns["ROCK TYPE"] == rock_type) & (selected_columns["SIO2(WT%)"] != 0)]
    tag_df['ALL_Alkaline']= tag_df["NA2O(WT%)"]+tag_df["K2O(WT%)"]

    # 检查是否存在tag_color_dict.json文件
    if os.path.exists('Color_Config/'+rock_type+'_color_dict.json'):
        # 如果存在，从文件中读取tag_color_dict
        with open('Color_Config/'+rock_type+'_color_dict.json', 'r') as f:
            tag_color_dict = json.load(f)
    else:
        # 如果不存在，创建新的tag_color_dict并保存到文件中
        type_set = set(tag_df['Type'].unique())
        cmap = cm.get_cmap('rainbow', len(type_set))
        tag_color_dict = {type: cmap(i) for i, type in enumerate(type_set)}
        with open('Color_Config/'+rock_type+'_color_dict.json', 'w') as f:
            json.dump(tag_color_dict, f) 


    # 输出'Type'的取值个数
    print(tag_df['Type'].value_counts())
    # 计算'Type'的取值个数
    type_counts = tag_df['Type'].value_counts()
    type_counts.to_csv(output_dir+'/TAS_'+rock_type+'_type_counts_tag.csv')

    # 绘制TAS图解散点图
    # label = tag_df['Type']
    # 假设df是包含'x', 'y', 'label'列的DataFrame
    labelled_groups = set()
    grouped = tag_df.groupby('Type')

    for label, group in grouped:
        ax.scatter(group["SIO2(WT%)"], group['ALL_Alkaline'], alpha=0.05, label=label, color=tag_color_dict[label], edgecolors='none')
        center_x = group["SIO2(WT%)"].mean()
        center_y = group['ALL_Alkaline'].mean()
        if label not in labelled_groups:
            if 35 <= center_x <= 80 and 0 <= center_y <= 17.6478:
                # ax.text(center_x, center_y, label, fontsize=9)
                pass
            else:
                print(label+" Coordinates are out of bounds")
                labelled_groups.add(label)

    # 绘制TAS图解边界线条
    # Draw TAS diagram boundary lines
    for line in cord['coords'].values():
        x_coords = [point[0] for point in line]
        y_coords = [point[1] for point in line]
        ax.plot(x_coords, y_coords, color='black', linewidth=0.3)
        

    # 在TAS图解中添加岩石种类标签
    # Add rock type labels in TAS diagram
    for label, coords in cord['coords'].items():
        x_coords = [point[0] for point in coords]
        y_coords = [point[1] for point in coords]
        x_center = sum(x_coords) / len(x_coords)
        y_center = sum(y_coords) / len(y_coords)
        ax.text(x_center, y_center, label, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.3), fontsize=12)

    ax.set_xlabel(r"SiO2", fontsize=16)
    ax.set_ylabel(r"Na2O+K2O", fontsize=16)
    ax.set_title(r"Extended TAS Diagram", fontsize=16)
    ax.set_xlim(35,80)
    ax.set_ylim(0,17.647826086956513)  

    ax.tick_params(axis='both', labelsize=12)
    legend = ax.legend(loc='upper left', fontsize=4, bbox_to_anchor=(1, 1))

    # 获取当前的视域范围
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # 计算在视域范围内的数据点的数量
    visible_points = tag_df[(tag_df["SIO2(WT%)"] >= xlim[0]) & (tag_df["SIO2(WT%)"] <= xlim[1]) & 
                            (tag_df['ALL_Alkaline'] >= ylim[0]) & (tag_df['ALL_Alkaline'] <= ylim[1])]

    num_visible_points = len(visible_points)

    # 在图上显示可见的数据点的数量
    ax.text(0.05, 0.95, f'Visible points: {num_visible_points}', transform=ax.transAxes, verticalalignment='top')

    fig.tight_layout()
    # 保存图，包含图例
    fig.savefig(output_dir+'/TAS_figure'+rock_type+'.svg')
    fig.savefig(output_dir+'/TAS_figure'+rock_type+'.png', dpi=600)
    # plt.show()

    conn.close()

# 文件名
filename = 'GeoRoc.db'
rock_type = 'VOL'
color_config_dir = 'Color_Config'
if not os.path.exists(color_config_dir):
    os.makedirs(color_config_dir)
output_dir = 'TAS'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

TAS_all(filename, 'VOL', output_dir = 'TAS')
TAS_all(filename, 'PLU', output_dir = 'TAS')

In [ ]:

def TAS_base(filename = 'GeoRoc.db',rock_type = 'VOL',output_dir='TAS'):

    result_list = [['Label','Probability']]
    
    # 创建一个指定宽高比的figure
    fig = plt.figure(figsize=(10, 10))     
    ax = fig.add_subplot(1, 1, 1)
    # 设置axes的宽高比为3:2
    # ax.set_aspect(2/3)

    # Record the start time
    begin_time = time.time()
    start_time = time.time()

    # 获取当前文件的绝对路径
    current_file_path = os.path.abspath(__file__)

    # 获取当前文件的目录
    current_directory = os.path.dirname(current_file_path)
    # 改变当前工作目录
    os.chdir(current_directory)


    with open(current_directory+'/Plot_Json/tas_cord.json', 'r', encoding='utf-8') as file:
        cord = json.load(file)


    # 连接到数据库
    conn = sqlite3.connect(filename)

    # Read the data from the TAS_Data table
    df = pd.read_sql_query("SELECT * FROM Current_Data", conn)

    selected_columns = df[["Type", "SIO2(WT%)", "NA2O(WT%)", "K2O(WT%)","ROCK TYPE"]]

    # 筛选'ROCK TYPE'为tag的行，并且去掉不含SiO2的行
    tag_df = selected_columns[(selected_columns["ROCK TYPE"] == rock_type) & (selected_columns["SIO2(WT%)"] != 0)]
    tag_df['ALL_Alkaline']= tag_df["NA2O(WT%)"]+tag_df["K2O(WT%)"]

    # 检查是否存在tag_color_dict.json文件
    if os.path.exists('Color_Config/'+rock_type+'_color_dict.json'):
        # 如果存在，从文件中读取tag_color_dict
        with open('Color_Config/'+rock_type+'_color_dict.json', 'r') as f:
            tag_color_dict = json.load(f)
    else:
        # 如果不存在，创建新的tag_color_dict并保存到文件中
        type_set = set(tag_df['Type'].unique())
        cmap = cm.get_cmap('rainbow', len(type_set))
        tag_color_dict = {type: cmap(i) for i, type in enumerate(type_set)}
        with open('Color_Config/'+rock_type+'_color_dict.json', 'w') as f:
            json.dump(tag_color_dict, f)    
    
    # 检查是否存在'TAS_Base_' + tag + '_Withlines.pkl'文件
    if os.path.exists(output_dir+'/'+'TAS_Base_' + rock_type + '_Withlines.pkl'):
        # 如果存在，从文件中读取tag_color_dict
        with open(output_dir+'/'+'TAS_Base_' + rock_type + '_Withlines.pkl', 'rb') as f:
            fig = pickle.load(f)
    else:
        pass

        # 输出'Type'的取值个数
        # print(tag_df['Type'].value_counts())
        # 计算'Type'的取值个数
        type_counts = tag_df['Type'].value_counts()
        type_counts.to_csv(output_dir+'/'+'TAS_type_counts_'+ rock_type +'_.csv')

        # 绘制TAS图解散点图
        # label = tag_df['Type']
        # 假设df是包含'x', 'y', 'label'列的DataFrame
        labelled_groups = set()
        grouped = tag_df.groupby('Type')

        label_locations = {}
        highest_y = 0

        for label, group in grouped:
            x = group["SIO2(WT%)"]
            y = group['ALL_Alkaline']
            center_x = x.mean()
            center_y = y.mean()
            
            if label not in labelled_groups:            
                labelled_groups.add(label)
                
                if 35 <= center_x <= 80 and 0 <= center_y <= 17.6478:
                    
                    data_amount = len(x)
                    # print(label, data_amount)
                    if(data_amount>30):

                        original_color =  mcolors.to_rgba(tag_color_dict[label])

                        # 定义一个基数，这个基数可以根据具体需求来调整
                        base = 0.08
                        # 计算透明度
                        alpha = base / np.log10(data_amount/10)             
                        
                        label_locations[label] = [center_x,center_y,original_color,alpha]
                        ax.scatter(x, y, color = original_color, edgecolors='none',  alpha = alpha)

                        # Record the end time
                        tmp_time = time.time()

                        # Calculate the time taken
                        time_taken = tmp_time - start_time
                        start_time = tmp_time

                        
                        print(f"{label} Data amount is {data_amount}, Alpha is {alpha:.3f}, Time taken: {time_taken:.3f} seconds")
                    
                        
                else:
                    # print(label+" Coordinates are out of bounds")
                    pass

        
        # 根据 center_x 对 label_locations 进行排序
        label_locations = dict(sorted(label_locations.items(), key=lambda item: item[1][0]))

        # 定义一个列表，包含n个值
        # 定义一个列表，步长为0.8
        values = list(np.arange(0, 7, 0.8))
        ha_list = ["left","right", "center", "left","right", "center"]
        va_list = ["bottom", "center", "top"]
        for i, (label, (center_x, center_y, original_color,alpha)) in enumerate(label_locations.items()):
            # 根据索引在 values 中轮流取值
            value = values[i % len(values)]
            ha = ha_list[i % len(ha_list)]
            va = va_list[i % len(va_list)]
            if(label =='Phonotephrite'):
                ha = 'left'
            # 在图中绘制文本
            used_x = center_x
            used_y = 18+value
            if highest_y <= used_y:
                highest_y = used_y
            ax.text(used_x, used_y, label, fontsize=11.5, color='k', 
                bbox=dict(facecolor=original_color, edgecolor=None, alpha= 0.3, pad=2),rotation = 0,
                horizontalalignment=ha, verticalalignment=va,
                            rotation_mode="anchor")

            # 添加一根从文本到中心点的虚线
            arrowstyle = '-|>'  # 箭头样式
            connectionstyle = ConnectionStyle("Arc3,rad=-0.3")  # 连接样式
            ax.annotate('', xy=(center_x, center_y), xytext=(used_x, used_y),
                        arrowprops=dict(arrowstyle=arrowstyle, connectionstyle=connectionstyle, linestyle='dashed', color=original_color, alpha=0.3))

        # 绘制TAS图解边界线条
        # Draw TAS diagram boundary lines
        for line in cord['coords'].values():
            x_coords = [point[0] for point in line]
            y_coords = [point[1] for point in line]
            ax.plot(x_coords, y_coords, color='black', linewidth=0.3)
            

        # 在TAS图解中添加岩石种类标签
        # Add rock type labels in TAS diagram
        for label, coords in cord['coords'].items():
            x_coords = [point[0] for point in coords]
            y_coords = [point[1] for point in coords]
            x_center = sum(x_coords) / len(x_coords)
            y_center = sum(y_coords) / len(y_coords)
            ax.text(x_center, y_center, label, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.3), fontsize=14)

        ax.set_xlabel("SiO2", fontsize=14)
        ax.set_ylabel("Na2O+K2O", fontsize=14)
        ax.set_title("Extended TAS Diagram", fontsize=14)
        ax.set_xlim(35,80)
        # ax.set_ylim(0,17.647826086956513)  
        # 在 y=17.647826086956513 的位置画一条横线
        ax.axhline(17.647826086956513, linestyle='-', color='black', linewidth=0.3)
        print(highest_y)
        ax.set_ylim(0,highest_y+1)  
        # 设置y轴的刻度
        ax.set_yticks(range(0, 18))

        ax.tick_params(axis='both', labelsize=12)
        # legend = ax.legend(loc='upper left', fontsize=4, bbox_to_anchor=(1, 1))

        # 获取当前的视域范围
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

        # 计算在视域范围内的数据点的数量
        visible_points = tag_df[(tag_df["SIO2(WT%)"] >= xlim[0]) & 
                                (tag_df["SIO2(WT%)"] <= xlim[1]) & 
                                (tag_df['ALL_Alkaline'] >= ylim[0]) & 
                                (tag_df['ALL_Alkaline'] <= ylim[1])]

        num_visible_points = len(visible_points)

        # 在图上显示可见的数据点的数量
        ax.text(0.05, 0.95, f'Used points: {num_visible_points}', transform=ax.transAxes, verticalalignment='top',fontsize=14)

        fig.tight_layout()

        with open(output_dir+'/'+'TAS_Base_' + rock_type + '_Withlines.pkl', 'wb') as f:
            pickle.dump(fig, f)

    all_end_time = time.time()

    all_time_taken = all_end_time - begin_time


    # 保存图，包含图例
    # 创建存图的文件夹
    fig.savefig(output_dir+'/'+'TAS_Base_' + rock_type + '.svg')
    # fig.savefig(output_dir+'/'+'TAS_Base_' + rock_type + '.pdf')
    fig.savefig(output_dir+'/'+'TAS_Base_' + rock_type + '.jpg', dpi=600)
    
    conn.close()
    print(f"All time taken: {all_time_taken:.3f} seconds")
    return(fig)
    

# 文件名
filename = 'GeoRoc.db'
rock_type = 'VOL'
color_config_dir = 'Color_Config'
if not os.path.exists(color_config_dir):
    os.makedirs(color_config_dir)
output_dir = 'TAS'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

TAS_base(filename, 'VOL', output_dir = 'TAS')
TAS_base(filename, 'PLU', output_dir = 'TAS')

In [ ]:

def Pearce_base(filename = 'GeoRoc.db',Type = 'Granite',output_dir='Pearce'):

    result_list = [['Label','Probability']]
    

    # Suppose you want a 2x2 grid of subplots
    fig, ax_list = plt.subplots(2, 2, figsize=(10, 10), sharex=False, sharey=False)

    plt.tight_layout()
    # 设置axes的宽高比为3:2
    # ax.set_aspect(2/3)

    # Record the start time
    begin_time = time.time()
    start_time = time.time()

    # 获取当前文件的绝对路径
    current_file_path = os.path.abspath(__file__)

    # 获取当前文件的目录
    current_directory = os.path.dirname(current_file_path)
    # 改变当前工作目录
    os.chdir(current_directory)


    with open(current_directory+'/Plot_Json/pearce_cord.json', 'r', encoding='utf-8') as file:
        cord = json.load(file)


    # 连接到数据库
    conn = sqlite3.connect(filename)

    # Read the data from the Pearce_Data table
    df = pd.read_sql_query("SELECT * FROM Current_Data", conn)

    Elements_List =["Y(PPM)", "NB(PPM)", "RB(PPM)", "YB(PPM)", "TA(PPM)"]
    # 筛选'Type'为Type的行，并且去掉数据不完整的行
    selected_columns = df[["Type", "TECTONIC SETTING", "Y(PPM)", "NB(PPM)", "RB(PPM)", "YB(PPM)", "TA(PPM)"]]
    conditions = (selected_columns["Type"] == Type)
    for element in Elements_List:
        conditions = conditions & (selected_columns[element] != 0)
    tag_df = selected_columns[conditions]



    def title_except_in_parentheses(s):
        parts = s.split('(')
        return parts[0].title() + '(' + parts[1].upper() if len(parts) > 1 else parts[0].title()

    tag_df = tag_df.rename(columns=title_except_in_parentheses)

    tag_df["Y+Nb(PPM)"] = tag_df["Y(PPM)"] + tag_df["Nb(PPM)"]
    tag_df["Yb+Ta(PPM)"] = tag_df["Yb(PPM)"] + tag_df["Ta(PPM)"]

    # print(tag_df)

    target_x_list = ['Y+Nb(PPM)','Yb+Ta(PPM)','Y(PPM)','Yb(PPM)']
    target_y_list = ['Rb(PPM)','Rb(PPM)','Nb(PPM)','Ta(PPM)']   
    limit_x_lit=[[0.1,2500],[0 , 300],[0 , 1000],[0.05 , 100]]
    limit_y_lit=[[0.5,9884],[1 , 7000],[0.5 , 2040],[0.05 , 100]]
    


    # 检查是否存在tag_color_dict.json文件
    if os.path.exists('Color_Config/'+Type+'_color_dict.json'):
        # 如果存在，从文件中读取tag_color_dict
        with open('Color_Config/'+Type+'_color_dict.json', 'r') as f:
            tag_color_dict = json.load(f)
    else:
        # 输出Tectonic Setting的取值个数
        print(tag_df["Tectonic Setting"].value_counts())
        # 如果不存在，创建新的tag_color_dict并保存到文件中
        type_set = set(tag_df["Tectonic Setting"].unique())
        cmap = cm.get_cmap('rainbow', len(type_set))
        tag_color_dict = {type: cmap(i) for i, type in enumerate(type_set)}
        with open('Color_Config/'+Type+'_color_dict.json', 'w') as f:
            json.dump(tag_color_dict, f)    
    
    # 检查是否存在'Pearce_Base_' + tag + '_Withlines.pkl'文件
    if os.path.exists(output_dir+'/'+'Pearce_Base_' + Type + '_Withlines.pkl'):
        # 如果存在，从文件中读取tag_color_dict
        with open(output_dir+'/'+'Pearce_Base_' + Type + '_Withlines.pkl', 'rb') as f:
            fig = pickle.load(f)
    else:
        pass

        
        # 绘制Pearce图解散点图
        # label = tag_df["Tectonic Setting"]
        # 假设df是包含'x', 'y', 'label'列的DataFrame
        labelled_groups = set()
        grouped = tag_df.groupby("Tectonic Setting")

        label_locations = {}
        highest_y = 0


        # 遍历ax_list和target_y_list
        for ax, target_x,target_y, limit_x,limit_y in itertools.zip_longest(ax_list.flatten(), target_x_list, target_y_list , limit_x_lit, limit_y_lit):

            for label, group in grouped:
                x = group[target_x]
                y = group[target_y]
                center_x = x.mean()
                center_y = y.mean()
                
                if label not in labelled_groups:            
                    labelled_groups.add(label)
                    
                data_amount = len(x)
                # print(label, data_amount)
                if(data_amount>30):

                    original_color =  mcolors.to_rgba(tag_color_dict[label])

                    # 定义一个基数，这个基数可以根据具体需求来调整
                    base = 0.08
                    # 计算透明度
                    alpha = base / np.log10(data_amount/10)             
                    
                    label_locations[label] = [center_x,center_y,original_color,alpha]

                    ax.scatter(x, y, color = original_color, edgecolors='none',  alpha = alpha)
                    ax.set_xlabel(target_x, fontsize=14)
                    ax.set_ylabel(target_y, fontsize=14)


                    # Remove NaN and Inf values
                    x = x[~np.isnan(x) & ~np.isinf(x)]
                    y = y[~np.isnan(y) & ~np.isinf(y)]

                    # # Check the number of unique values
                    # if len(np.unique(x)) > 1:
                    #     x_lower, x_upper = np.percentile(x, [5, 95])
                    # else:
                    #     x_lower, x_upper = x[0] - 1, x[0] + 1

                    # if len(np.unique(y)) > 1:
                    #     y_lower, y_upper = np.percentile(y, [5, 95])
                    # else:
                    #     y_lower, y_upper = y[0] - 1, y[0] + 1

                    # # Set the limits of x and y axes
                    # ax.set_xlim(x_lower, x_upper)
                    # ax.set_ylim(y_lower, y_upper)

                    ax.set_xlim(limit_x[0],limit_x[1])
                    ax.set_ylim(limit_y[0],limit_y[1])                
                    ax.set_xscale('log')
                    ax.set_yscale('log')
                    ax.set_aspect('equal') 
                    ax.tick_params(axis='both', labelsize=12)    
                    # 旋转y轴的标签
                    ax.yaxis.set_tick_params(rotation=90)

                    # Record the end time
                    tmp_time = time.time()

                    # Calculate the time taken
                    time_taken = tmp_time - start_time
                    start_time = tmp_time
                    
                    print(f"{label} Data amount is {data_amount}, Alpha is {alpha:.3f}, Time taken: {time_taken:.3f} seconds")      
                                                 

                try:
                    # 获取当前ax对象中的所有数据点
                    for child in ax.get_children():
                        # 检查这个子对象是否是一个散点图的集合
                        if isinstance(child, collections.PathCollection):
                            # 获取当前透明度
                            current_alpha = child.get_alpha()
                            # 获取数据点的数量
                            num_points = child.get_sizes().size
                            # 根据当前透明度和数据点的数量设置新的透明度
                            if current_alpha is not None:
                                if num_points <1000:  # 如果数据点的数量大于100
                                    child.set_alpha(min(current_alpha * 2, 0.3))  # 提高透明度，但不超过1
                                elif num_points >3000:  # 如果数据点的数量小于50
                                    child.set_alpha(max(current_alpha / 2, 0.005))  # 降低透明度，但不低于0.01

                except KeyError:
                    pass
        
            # 根据 center_x 对 label_locations 进行排序
            label_locations = dict(sorted(label_locations.items(), key=lambda item: item[1][0]))

            # # 定义一个列表，包含n个值
            # # 定义一个列表，步长为0.8
            # values = list(np.arange(0, 7, 0.8))
            # ha_list = ["left","right", "center", "left","right", "center"]
            # va_list = ["bottom", "center", "top"]
            # for i, (label, (center_x, center_y, original_color,alpha)) in enumerate(label_locations.items()):
            #     # 根据索引在 values 中轮流取值
            #     value = values[i % len(values)]
            #     ha = ha_list[i % len(ha_list)]
            #     va = va_list[i % len(va_list)]
            #     # 在图中绘制文本
            #     used_x = center_x
            #     used_y = 18+value
            #     if highest_y <= used_y:
            #         highest_y = used_y
            #     ax.text(used_x, used_y, label, fontsize=11.5, color='k', 
            #         bbox=dict(facecolor=original_color, edgecolor=None, alpha= 0.3, pad=2),rotation = 90,
            #         horizontalalignment=ha, verticalalignment=va,
            #                     rotation_mode="anchor")

            #     # 添加一根从文本到中心点的虚线
            #     arrowstyle = '-|>'  # 箭头样式
            #     connectionstyle = ConnectionStyle("Arc3,rad=-0.3")  # 连接样式
            #     ax.annotate('', xy=(center_x, center_y), xytext=(used_x, used_y),
            #                 arrowprops=dict(arrowstyle=arrowstyle, connectionstyle=connectionstyle, linestyle='dashed', color=original_color, alpha=0.3))

            # # 绘制Pearce图解边界线条
            # # Draw Pearce diagram boundary lines
            # for line in cord['coords'].values():
            #     x_coords = [point[0] for point in line]
            #     y_coords = [point[1] for point in line]
            #     ax.plot(x_coords, y_coords, color='black', linewidth=0.3)
            

            # # 在Pearce图解中添加岩石种类标签
            # # Add Type labels in Pearce diagram
            # for label, coords in cord['coords'].items():
            #     x_coords = [point[0] for point in coords]
            #     y_coords = [point[1] for point in coords]
            #     x_center = sum(x_coords) / len(x_coords)
            #     y_center = sum(y_coords) / len(y_coords)
            #     ax.text(x_center, y_center, label, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.3), fontsize=14)


            # ax.set_title("Extended Pearce Diagram", fontsize=14)
            # ax.set_xlim(35,80)
            # ax.set_ylim(0,17.647826086956513)  
            # # 在 y=17.647826086956513 的位置画一条横线
            # ax.axhline(17.647826086956513, linestyle='-', color='black', linewidth=0.3)
            # print(highest_y)
            # ax.set_ylim(0,highest_y+1)  
            # # 设置y轴的刻度
            # ax.set_yticks(range(0, 18))

            # legend = ax.legend(loc='upper left', fontsize=4, bbox_to_anchor=(1, 1))

            # # 获取当前的视域范围
            # xlim = ax.get_xlim()
            # ylim = ax.get_ylim()

            # 计算在视域范围内的数据点·的数量
            # visible_points = tag_df[(tag_df["Y(PPM)"] >= xlim[0]) & 
            #                         (tag_df["Y(PPM)"] <= xlim[1]) & 
            #                         (tag_df['ALL_Alkaline'] >= ylim[0]) & 
            #                         (tag_df['ALL_Alkaline'] <= ylim[1])]

            # num_visible_points = len(visible_points)
            num_visible_points = len(tag_df)

            # 在图上显示可见的数据点的数量
            ax.text(0.05, 0.95, f'Used points: {num_visible_points}', transform=ax.transAxes, verticalalignment='top',fontsize=14)

        fig.tight_layout()
            

        with open(output_dir+'/'+'Pearce_Base_' + Type + '_Withlines.pkl', 'wb') as f:
            pickle.dump(fig, f)

    all_end_time = time.time()

    all_time_taken = all_end_time - begin_time


    # 保存图，包含图例
    # 创建存图的文件夹
    fig.savefig(output_dir+'/'+'Pearce_Base_' + Type + '.svg')
    # fig.savefig(output_dir+'/'+'Pearce_Base_' + Type + '.pdf')
    fig.savefig(output_dir+'/'+'Pearce_Base_' + Type + '.jpg', dpi=600)
    
    conn.close()
    print(f"All time taken: {all_time_taken:.3f} seconds")
    return(fig)
    

# 文件名
filename = 'GeoRoc.db'
Type = 'Granite'
color_config_dir = 'Color_Config'
if not os.path.exists(color_config_dir):
    os.makedirs(color_config_dir)
output_dir = 'Pearce'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Pearce_base(filename, 'Granite', output_dir = 'Pearce')